In [1]:
#| default_exp routing


In [2]:
#| export
from __future__ import annotations


# Routes provider

> Enables method-based routing in fasthtml by extending `APIRouter` capabilities.  

FastHTML currently supports two scopes for route endpoints: global and local. Here, I proppose a third namespace, class.

This allows objects to provide routes through their methods, similar to class-based views in other web frameworks.


# Prologue

In [3]:
#| export

import functools
from contextlib import contextmanager
from functools import update_wrapper
from types import FunctionType
from types import MethodType
from typing import Any
from typing import Callable
from typing import overload
from typing import Protocol
from typing import Sequence

import fasthtml.core
from fasthtml.core import all_meths
from fasthtml.core import APIRouter
from fasthtml.core import FastHTML
from fasthtml.core import noop_body


In [4]:
#| export

from bridget.helpers import id_gen
from bridget.display_helpers import nb_app


In [5]:
import inspect
import typing
from dataclasses import dataclass
from functools import partial
from itertools import groupby
from typing import cast
from typing import ClassVar
from typing import Iterable
from typing import Mapping
from typing import Self
from typing import Type

import fastcore.all as FC
import starlette.routing
import starlette.types
from fastcore.test import *
from fasthtml.components import Button
from fasthtml.components import Div
from fasthtml.core import uri
from rich.console import Console
from starlette.responses import HTMLResponse
from starlette.responses import Response
from starlette.routing import Match
from starlette.routing import Route
from starlette.testclient import TestClient


----


In [6]:
#| exporti

new_id = id_gen()


In [7]:
cprint = (console := Console(width=120)).print


----

In [8]:
def get_app():
    return (app := nb_app()), TestClient(app, base_url='http://nb', headers={'hx-request': '1'}), app.route

app, cli, rt = get_app()

# patched
> Quick hack to monkey-patch a module function.

`FastHTML` only considers as of now (v.0.12.0) names of static (possibly nested) functions. We'll need to temporarily patch `nested_name` to get right the name of methods (see `APIRouterD` below).


In [9]:
#| exporti

@contextmanager
def _patched(o, nm, f):
    original = getattr(o, nm)    
    setattr(o, nm, update_wrapper(f, original))
    try: yield
    finally: setattr(o, nm, original)


# Methods as routes endpoints
> Understanding FastHTML's routing system and extending it to support instance methods.

:::{.callout-note}
In this docs I use widget/component interchangeably.
:::



### Why Method-Based Routes in Notebooks?

FastHTML typically encourages defining components in separate modules. While this works great for web applications, notebooks have different workflows:

**Interactive Development**  
Notebooks are all about exploration and rapid prototyping. Everything happens in cells - they're our unit of work. Notebooks are more akin to one page apps. When you're quickly testing ideas or building one-time components, creating separate modules can feel like unnecessary overhead. Method-based routes let us define and test components right where we need them, with scoping and encapsulation.

**State Management**  
When building widgets, we often need to maintain state. Instance methods make this natural. While both modules and classes can handle state effectively, sometimes one approach fits better than the other. It's nice to have options.

**Quick Iteration**  
The real advantage comes when you're iterating on a component. You can write code, test it, and modify it all in the same context. Make a change, run the cell, see the results. No jumping between files or reloading modules needed.

This approach may complement FastHTML's module-based components by providing another option for route management.

# Scoping

:::{.callout-note}
All this discussion is for current FastHTML, v0.12.0 at the time of this writing. FastHTML is young and nervous, so things may change quickly and unexpectedly.
:::


In [10]:
app, cli, rt = get_app()

def a(): return 'a'  # type: ignore
test_eq(type(a), FunctionType)
test_eq(str(a), f"<function a at 0x{id(a):x}>")

global_a = a


# @rt('/a', 'get')
# def a(): return 'a'

# or its nearly equivalent:
a = rt('/a', 'get')(a)

assert a is not global_a
test_eq(str(a), '/a')
test_eq(cli.get('/a').text, 'a')
test_eq(app.url_path_for('a'), '/a')
test_eq(a.to(), '/a')
test_eq(set(app.routes[-1].methods), {'GET', 'HEAD'})


@rt('/a', 'post')
def a(x:int): return f'a {x}'

test_eq(cli.post('/a?x=5').text, 'a 5')
test_eq(set(app.routes[-1].methods), {'POST'})


`FastHTML.route` decorator, or the common `rt` alias, creates a Starlette `Route` with an endpoint based on a static function, function `a` in this case. But it's not function `a`, it's a especial callable wrapped over the function `a` to facilitate route introspection.

During route definition, `a` is removed from the scope it was defined and can be used to create additional routes with other HTTP methods (with caveats, `'post'` is a especial case).


In [11]:
def b():
    @rt('/b')
    def _b(): return 'b'
    return _b

innerb = b()
test_eq(cli.get('/b').text, 'b')
test_eq(str(innerb), '/b')
test_eq(app.url_path_for('b__b'), '/b')
test_eq(innerb.to(), '/b')


Recent FastHTML versions also supports nested functions on local scopes.


In [12]:
ar = APIRouter()

@ar('/c')
def c(): return 'c'

test_eq(str(c), '/c')
test_eq(ar.c.to(), '/c')
test_eq(ar.rt_funcs.c.to(), '/c')

ar.to_app(app)

test_eq(cli.get('/c').text, 'c')
test_eq(app.url_path_for('c'), '/c')


`APIRouter` is a convenient way of grouping routes.

### FastHTML's Route Requirements

FastHTML's routing system builds on Starlette's, which accepts both functions and methods as endpoints (besides ASGI classes). The only requirement is being a [callable](https://github.com/encode/starlette/blob/427a8dcf357597df27b2509b1ac436caf7708300/starlette/routing.py#L208):


```python
endpoint: typing.Callable[..., typing.Any]
```

FastHTML follows FastAPI's style of using decorators for route definition. Route endpoints must be:
1. Callables with a `__name__` (`__qualname__` as of v0.12.0) attribute (for route identification) [FastHTML._add_route](https://github.com/AnswerDotAI/fasthtml/blob/bff83a9bca000c38c91909809e2726a51418793f/fasthtml/core.py#L608)
2. Have type annotations for all parameters (for request validation)

While this works for functions, it creates challenges with instance methods:

1. **Method Binding**: When using decorators at class level, methods are still unbound functions:
   - The decorator sees the function `get`, not the bound method
   - The `self` parameter won't be properly valued

2. **Type Annotations**: FastHTML expects all parameters to be annotated:
   - The implicit `self` parameter lacks annotation
   - This triggers warnings in FastHTML's validation

`APIRouter` like `rt` only works wth static functions. 

As a workaround we could use `partial` (or FastHTML own trick, a lightweight stateful callable) to bind methods after instance creation. As partials don't have a `__name__` (`__qualname__` now) attribute, we need to hack it manually:


In [13]:
class AClass:
    def get(self, a:str): return f'{self!r} {a!r}'

a = AClass()
# Manually create bound method and set name
(f := partial(a.get.__func__, a)).__qualname__ = 'get'  # type: ignore
rt('/give/me/{a}')(f)

cli.get('/give/me/b').text


"<__main__.AClass object at 0x10fdc55b0> 'b'"

This approach is verbose and error-prone, separates route definition from method implementation, and makes code harder to maintain, besides being cumbersome and ugly. We need something more in order to use methods as endpoints with fastHTML, clearly time for some more python dark magic here. But we're in python-land where magic abounds.

We'll extend FastHTML's routing capabilities by:
- Creating an enhanced `APIRouter` class that support (and preserves) method references
- Supporting automatic method binding at instance creation
- Enabling property-based routing for GET/POST/PUT/DELETE operations
- Providing automatic route mounting and path generation

We aim for a more natural and maintainable syntax:

```python
class Widget:
    ...
    @ar
    def value(self): 
        return self._value
```

We'll use the handy `APIRouter` to define routes providers.

:::{.callout-note}
Most of this notebook was written before v0.12.0. In v0.9.1, `APIRouter` was defined but not used anywhere. I wasn't sure what were Jeremy's intention for the class, but it seemed appropiate using it here for our purposes.

From v0.10.2 to v0.12.0, `APIRouter` has evolved considerably but the basic idea proposed here holds through.
:::


First, some helpers to get the routes from a provider and its class hierarchy.

In [14]:
def routes_from(o: object):
    "Yield all route descriptors (path,methods,name,include_in_schema,body_wrap) from class hierarchy in mro order"
    for base in (o if isinstance(o, type) else type(o)).mro()[:-1]:  
        if not isinstance(base, type) or not hasattr(base, 'ar'): continue
        yield from base.ar.routes

def add_routes(app: FastHTML, o):
    for f,p,m,n,*_ in routes_from(o):
        app.add_route(Route(p, f, methods=m, name=n))


In [15]:
class A:
    ar = APIRouter()
    @ar('/a')
    def a(self): return 'a'

class B(A):
    ar = APIRouter()
    @ar('/b')
    def b(self): return 'b'

class C(B):
    ar = APIRouter()
    @ar('/c?v={v}')
    def c(self, v:int=0): return f'c {v}'

class D(A):
    ar = APIRouter()
    @ar('/d')
    def d(self): return 'd'

class E(B, D):
    ar = APIRouter()
    @ar('/a')
    def e(self): return 'e'


test_eq(C.mro()[:-1], [C, B, A])
test_eq(E.mro()[:-1], [E, B, D, A])

test_eq([_[1] for _ in list(routes_from(C))],  ['/c?v={v}', '/b', '/a'])
test_eq([_[1] for _ in list(routes_from(E))],  ['/a', '/b', '/d', '/a'])


In [16]:
app, cli, _ = get_app()
add_routes(app, C)

test_eq([_.path for _ in app.routes], ['/c?v={v}', '/b', '/a'])

c_rt = app.routes[0]

match, _ = c_rt.matches({'type': 'http', 'path': '/c?v=3', 'method': 'GET'})
test_eq(match, Match.FULL)

match, _ = c_rt.matches({'type': 'http', 'path': '/c?v=7', 'method': 'PUT'})
test_eq(match, Match.PARTIAL)


In [17]:
app, cli, _ = get_app()
add_routes(app, E)

test_eq([_.path for _ in app.routes], ['/a', '/b', '/d', '/a'])

e_rt = app.routes[0]

match, child_scope = e_rt.matches({'type': 'http', 'path': '/a', 'method': 'GET'})
test_eq(match, Match.FULL)
test_eq(child_scope['endpoint'], e_rt.endpoint)
test_eq(e_rt.endpoint(''), 'e')


`provider_routes` returns the routes in MRO order. If we add the routes in that order, Starlette will match them in that order too (NOTE: check this is documented). As such, more specific endpoints have precedence over more ones.

Starlette accepts methods as endpoints with no probles, but as we've seen, not current FastHTML versions (0.12.0 as of this writing). In FastHTML, we need to use bound functions -- AKA methods--. And that is a bit more involved than using `app.route` on static functions.


# RouteProvider
> Any object with an `APIRouter` attribute can be a provider of routes.


In [18]:
#| export

_AR_MARK = '__routes__'

# @typing.runtime_checkable
class RouteProviderP(Protocol): 
    __routes__: Sequence  # of (func, path, methods, name, include_in_schema, body_wrap) like `APIRouter`'s routes attr.
    ar: APIRouter

class RouteProvider:  # noop class, only to alliviate the stupid wiggly reds
    __routes__: Sequence
    ar: APIRouter


# APIRouterD
> APIRouter as a descriptor

APIRouter for Method-Based Routes.  

Extends `FastHTML`'s `APIRouter` to preserve method references while enabling route registration.

~~For reasons unclear to me, APIRouter wipes out the functions when defining routes unlike normal `@rt` decorator.~~

We need to keep the functions around in order for the class to get its methods.

`MethodsRouter` preserves the original method references while collecting routes. This allows methods to work both as routes and methods, with proper instance binding. Like FastHTML's standard APIRouter, routes are stored but not registered immediately, which lets us bind them to instances at mount time. This delayed registration enables stateful components and works naturally with Python properties and methods, maintaining compatibility with all FastHTML's routing features.


In [19]:
#| exporti

def nested_name(f):
    "Get name of function `f` using '_' to join nested function names"
    return f.__qualname__.replace('.<locals>.', '_').replace('.', '_')


In [20]:
def f():
    def ff(): return 'ff'
    return ff

test_eq(f.__name__, 'f')
test_eq(f.__qualname__, 'f')
test_eq(f().__name__, 'ff')
test_eq(f().__qualname__, 'f.<locals>.ff')
test_eq(nested_name(f), 'f')
test_eq(nested_name(f()), 'f_ff')

class A:
    def f(self):
        def ff(): return 'ff'
        return ff

test_eq(A.f.__name__, 'f')
test_eq(A.f.__qualname__, 'A.f')
test_eq(A.f(A()).__name__, 'ff')
test_eq(A.f(A()).__qualname__, 'A.f.<locals>.ff')
test_eq(nested_name(A.f), 'A_f')
test_eq(nested_name(A.f(A())), 'A_f_ff')


In [21]:
class A:
    def f(self):
        def ff(): return 'ff'
        return ff


with _patched(fasthtml.core, 'nested_name', nested_name):
    test_eq(fasthtml.core.nested_name(A.f), 'A_f')
    test_eq(fasthtml.core.nested_name(A.f(A())), 'A_f_ff')


In [22]:
#| exporti

# def provider_routes(prov: object):
#     "Yield all route specs (func,path,methods,name,include_in_schema,body_wrap) from class hierarchy in mro order"
#     for base in (prov if isinstance(prov, type) else type(prov)).mro()[:-1]:  
#         if not isinstance(base, type) or not hasattr(base, _AR_MARK): continue
#         yield from getattr(base, _AR_MARK).items()

def _fn(f):
    return f.fget.__name__ if isinstance(f, property) else f.__name__  # type: ignore

def _path(path):
    if isinstance(path, str): return path
    if isinstance(path, property): return f"/{getattr(path.fget, '__name__')}"
    if callable(path) or isinstance(path, classmethod): return f"/{path.__name__}"
    raise TypeError(f"Invalid path type: {type(path)}")

_prop2mth = list(zip(('fget', 'fset', 'fdel'), (['GET'], ['POST', 'PUT'], ['DELETE'])))

def _bound(func, o):
    if isinstance(func, FunctionType): return MethodType(func, o)
    if isinstance(func, (staticmethod, classmethod)): return getattr(o, func.__name__)
    if isinstance(func, classmethod): return getattr(type(o), 'd')
    raise TypeError(f"Invalid function type: {type(func)}")

# def _as_mths(func, o):
#     if isinstance(o, type):
#         if isinstance(func, property):
#             yield from ((f, m) for fn, m in _prop2mth if (f := getattr(func, fn, None)))
#         else: yield func, None
#     else:
#         if isinstance(func, property):
#             yield from ((MethodType(f, o), m) for fn, m in _prop2mth if (f := getattr(func, fn, None)))
#         # else: yield getattr(o, func.__name__), None
#         elif isinstance(func, (staticmethod, classmethod)): yield getattr(o, func.__name__), None
#         else: yield MethodType(func, o), None
def _unravel_prop(func, o):
    if isinstance(func, property):
        yield from ((f, m) for fn, m in _prop2mth if (f := getattr(func, fn, None)))
    else: yield func, None
def _reg_mth(ar, func, args, o):
    for mth, m in _unravel_prop(func, o):
        path, methods, *_ = args
        ar(path, methods or m or 'get', *_)(mth)


I think the original intuition of Jeremy with APIRouter was correct: a simple class that gathered the routes arguments and defined the final routes in to_app. But as it removed the functions from the definition scope, it was not possible to inspect the routes with usual to() or reverse lookup URLs.

The current version solves the issue adding an rt_funcs attribute to APIRouter and adding the routes as attributes.


## APIRouterC

In [23]:
#| exporti

class APIRouterC(APIRouter):

    def _wrap_func(self, func, path=None):
        name = func.__name__
        wrapped = fasthtml.core._mk_locfunc(func, path)
        wrapped.__routename__ = nested_name(func)
        # If you are using the def get or def post method names, this approach is not supported
        if name not in all_meths: setattr(self.rt_funcs, name, wrapped)
        return wrapped

    # Don't like monkey-patching `core`, let's see if transient patching is enough
    def to_app(self, app): 
        "Add routes to `app`"
        with _patched(fasthtml.core, 'nested_name', nested_name):
            for args in self.routes: app._add_route(*args)
            for args in self.wss: app._add_ws(*args)
    
    def _rts(self, fn): return list(filter(lambda x: x[0].__name__==fn, self.routes))

    def to(self, fn=None, /, **kw) -> str: 
        if fn is None: return self.prefix.strip('/')
        if not (rr := self._rts(fn)): raise NameError(f"Endpoint {fn} not found.")
        return fasthtml.core.qp(f"{self.prefix.strip('/')}{rr[0][1]}", **kw)  # type: ignore

    def name(self, fn=None) -> str: 
        if fn is None: return getattr(self, '__routename__', '')
        if not (rr := self._rts(fn)): raise NameError(f"Endpoint {fn} not found.")
        # return nested_name(rr[0][0])
        return f"{(nm:=self.name())+(':' if nm else '')}{nested_name(rr[0][0])}"


`fasthtml.corenested_name` only considers static (nested) function.

Current APIRouter (v0.12.0) has a minor issue in _wrap_func, it uses `__name__` for the name of the route instead of `__qualname__`. I could patch it, but this subclass will do for now. 

APIRouterC also has an alternative to rt_funcs, given the method name:
  - APIRouterC.to(name, ...) return the route path.
  - APIRouterC.name(name) return the route name.
  - can use HTTP methods as method names


## APIRouterD

In [24]:
#| export

class APIRouterD(APIRouterC):
    def __init__(self, *args, **kwargs): self._rtargs, self._name = {}, None; super().__init__(*args, **kwargs)
    
    @overload
    def __call__(self, path:FunctionType, *args, **kwargs) -> Callable: ...
    @overload
    def __call__(self, path:classmethod, *args, **kwargs) -> classmethod: ...
    @overload
    def __call__(self, path:property, *args, **kwargs) -> property: ...
    @overload
    def __call__(self, path:str, *args, **kwargs) -> Callable: ...
    def __call__(self, path:Any=None, methods=None, name=None, 
                        include_in_schema=True, body_wrap=noop_body):
        # gather precursors of routes
        def f(func):
            p = _path(path)
            self._rtargs[func] = '/' if p == '/index' else p, methods, name, include_in_schema, body_wrap
            return func  # let methods be methods
        return f if path is None or isinstance(path, str) else f(path)

    def __set_name__(self, owner, name): 
        assert not self._name, f"APIRouterD already named {self._name}."
        self._name = name
        for func, args in self._rtargs.items(): 
            for mth, m in _unravel_prop(func, owner):
                path, methods, *rest = args
                self.routes.append((mth, path, methods or m or 'get', *rest))
        for base in owner.mro()[1:-1]: 
            if rr := getattr(base, _AR_MARK, None):
                self.routes.extend(_ for _ in rr if _ not in self.routes)
        setattr(owner, _AR_MARK, self.routes)  # mark class as routes provider

    def __get__(self, instance, owner):
        # bind my routes to class/instance
        if not self._name: # shouldn't happen
            raise AttributeError(f"APIRouterD {self._name} not bound to {owner}")
        if not instance: return self
        ar = APIRouterC(self.prefix, self.body_wrap) # final routes w/ bounded endpoint
        for func, *args in self.routes: ar(*args)(_bound(func, instance))
        setattr(instance, self._name, ar)  # type: ignore # instance doesn't need the descriptor any more
        return ar


    def to_app(self, app): raise TypeError("APIRouterD can't add routes to an app")

for o in all_meths: setattr(APIRouterD, o, functools.partialmethod(APIRouterD.__call__, methods=o))


In [25]:
# type: ignore

class A:
    ar = APIRouterD()

    def __init__(self, value:int=3): self.value = value

    @ar  # path /, name A_index
    def index(self):
        # int 0 is not a valid FastHTML return value
        return self.value if self.value != 0 else HTMLResponse('0')

    @ar  # or @ar.get or @ar.get('/a')
    def a(self): return f'a {self.value}'

    @ar.post('/a')
    def a_change(self, x:int): self.value = x; return f'new value {x}'

    @ar('/b')  # name A_b, , method [GET, POST]
    def b(self): return f'b {self.value}'

    @ar  
    @staticmethod
    def c(): return 'c'

    @ar
    @classmethod
    def d(cls): return f'd {cls.__name__}'

    @ar
    @property  # path /e, name A_e, method GET
    def e(self): return f'e {self.value}'

    @ar
    @e.setter  # path /e, name A_e, method [POST, PUT]
    def e(self, x:int): self.value = x; return x

    @ar  # path /e, name A_e, method DELETE
    @e.deleter
    def e(self): del self.value; return Response(status_code=204)

app, cli, _ = get_app()

a = A()
a.ar.to_app(app)

test_eq(cli.get('/').text, '3')

test_eq(cli.get('/a').text, 'a 3')
test_eq(app.url_path_for('A_a'), '/a')
test_eq(a.ar.a.to(), '/a')
test_eq(a.ar.to('a'), '/a')
test_eq(set([*filter(lambda r: r.path == '/a', app.routes)][0].methods), {'GET', 'HEAD'})

test_eq(cli.post('/a?x=5').text, 'new value 5')
test_eq(a.value, 5)

test_eq(cli.get('/b').text, 'b 5')
test_eq(cli.get('/c').text, 'c')
test_eq(cli.get('/d').text, 'd A')

test_eq(cli.get('/e').text, 'e 5')
test_eq(cli.post('/e?x=7').text, '7')
test_eq(a.value, 7)
test_eq(str(cli.delete('/e')), '<Response [204 No Content]>')

a.value = 0
test_eq(cli.get('/').text, '0')


Note that we can also set properties as routes. Use as above, or simply once at last descriptor function.

In [26]:
class A:
    ar = APIRouterD()

    def __init__(self, value:int=3): self.value = value

    @property
    def e(self): return f'e {self.value}'

    @e.setter
    def e(self, x:int): self.value = x; return x

    @ar
    @e.deleter
    def e(self): del self.value; return Response(status_code=204)


app, cli, _ = get_app()

a = A()
a.ar.to_app(app)

test_eq(cli.post('/e?x=3').text, '3')
test_eq(a.value, 3)
test_eq(str(cli.delete('/e')), '<Response [204 No Content]>')


Note that the APIRouter of the class, `A.ar` in the example above, should be used only for route inspection. The one on the instance, `a.ar`, is the who actually can registers the routes and is in fact a normal `APIRouter`.

An `APIRouterD route`is very alike `FastHTML.route` or `APIRouter`'s. But as its underlying endpoint is a method/property, it has some different behaviors:


## route names

In [27]:
app, cli, rt = get_app()

@rt('/a')
def a(): return 'a'

print(f"{'route from a ->':>20}", app.routes[-1])
test_eq(a.to(), app.url_path_for('a'))
test_eq(a.__routename__, 'a')

def b():
    @rt('/a')
    def c(): return 'c'
    return c

c = b()

print(f"{'route from c ->':>20}", app.routes[-1])
test_eq(c.to(), app.url_path_for('b_c'))
test_eq(c.__routename__, 'b_c')

class D:
    ar = APIRouterD()
    @ar('/a')
    def d(self): return 'd'

d = D()
d.ar.to_app(app)

print(f"{'route from B.b ->':>20}", app.routes[-1])
test_eq(d.ar.d.to(), app.url_path_for('D_d'))
test_eq(d.ar.d.__routename__, 'D_d')

test_eq(d.ar.to('d'), d.ar.rt_funcs.d.to())
test_eq(d.ar.name('d'), 'D_d')


     route from a -> Route(path='/a', name='a', methods=['GET', 'HEAD', 'POST'])
     route from c -> Route(path='/a', name='b_c', methods=['GET', 'HEAD', 'POST'])
   route from B.b -> Route(path='/a', name='D_d', methods=['GET', 'HEAD'])



- the name is the method name, `A_a` in the example above, not `a`.
- default method is 'get'.
- **Route Introspection**: Methods, like functions used in routes, get the .to() attribute, discoverable through `ar`.
- unlike `@rt/APIRouter`, it doesn't wipe out (_add_route) the underlying function from the scope (class def) for obvious reasons-- base methods would probably be ok, properties not. Python classes can't have methods with the same name-- dicts, you know.  
  If you want to handle different HTTP methods, you must use different func names or set them explcitly with the route arguments and modify the function to handle the distinct requests (args or explicit request arg).

:::{.callout-note}
Last point is a design decission. 
We could use metaclasses like FascCore transforms to introduce methods overriden by HTTP methods and in theory _mk_locfunc wrappers could function with methods (probably, not tested much). But I think a descriptor is already enough dark python magic for today :)
:::

In [28]:
test_eq(A.ar.to(), '')
test_eq(A.ar.to('e'), '/e')
with ExceptionExpected(AttributeError): A.ar.a
with ExceptionExpected(AttributeError): A.ar.rt_funcs.a

with ExceptionExpected(TypeError): A.ar.to_app(app)


#  Inheritance

As usual with OOP and classes, let's say its complicated.

In [29]:
# type: ignore

class A:
    ar = APIRouterD('/A')

    @ar
    def value(self): return self._value

class B(A):
    ar = APIRouterD('/AB')

    @ar.post
    def value(self, x:int):
        self._value = x
        return self._value

@dataclass
class C(B):
    ar = APIRouterD('/ABC')
    _value:int = 1


app, cli, _ = get_app()

c = C()
c.ar.to_app(app)

test_is(B.value, C.value)
with ExceptionExpected(TypeError): c.value()

test_eq(cli.get('/ABC/value').text, '1')  # <-- equivalent to A.value(c)
test_eq(cli.post('/ABC/value?x=5').text, '5')  # <-- equivalent to B.value(c, x=5)
test_eq(A.value(c), 5)


You must be careful mixing inheritance and Starlette routing. 

Method resolution in python follows [MRO](https://docs.python.org/3/howto/mro.html#python-2-3-mro). Starlette routes lay in a flattened space, a list (or list of lists if mounting apps). Unlike python, routes are matched in the order they are added by path, path parmas and HTTP methods.

In the example above, python call fails, but Starlette has no problem finding the route. A Route endpoint is a function, unbounded or, if using APIRouterD, bounded to the instance (of type C, not B or A, in the example). FastHTML adds extended functionality such as typing and introspection. The call works, but not in a pythonic way.

Without getting into (Liskov)[https://en.wikipedia.org/wiki/Liskov_substitution_principle] and subtyping and the rest of OOP can of worms, think of APIRouterD as a way of organize routes and encapsulate state, nothing more. In Hypermedia systems, inheritance, classes, typing are ill defined concepts if at all. There's not such thing as a "subclass" of a resource. Resources are uniquely idewntified with URIs, oprganized in hypermedia networks obtained with hypermedia controls. Semantics or behaviors are a thing of the server. The client only knows URIs.

So, stick with simple, flat scopes, and if you must, be very careful how you define the routes when inheritance is involved.


# APIRoute
> Convenience `APIRouterD` shortcut


In [30]:
#| export

class _APIRouterD(APIRouterD):
    def __set_name__(self, owner, name): raise AttributeError(f"_APIRouterD should not be used explicitly.")
    def __get__(self, instance, owner):
        if not instance: return self
        for base in reversed(owner.mro()[1:-1]):
            if hasattr(base, _AR_MARK):
                _ar = getattr(base, 'ar', None)
                if _ar and not _ar._name: super(_APIRouterD, _ar).__set_name__(base, 'ar')
        if not self._name: super().__set_name__(owner, 'ar')    
        return super().__get__(instance, owner)


class APIRoute:
    _attr_name = 'ar'

    def __init__(self, path=None, methods=None, name=None, include_in_schema=True, body_wrap=noop_body):
        if path and not isinstance(path, str): self.func, path = path, _path(path)
        self._rtargs = '/' if path == '/index' else path, methods, name, include_in_schema, body_wrap

    def __call__(self, func) -> Callable:  self.func = func; return self
    def __getattr__(self, name): return getattr(self.func, name)  # getter, setter, deleter
    
    def __set_name__(self, owner, name):
        if not self._attr_name in vars(owner):
            setattr(owner, self._attr_name, ar := _APIRouterD())
            setattr(owner, _AR_MARK, ar._rtargs)
        getattr(owner, _AR_MARK)[self.func] = self._rtargs  # collect rt args
        setattr(owner, name, self.func)  # let methods be methods; wipe out myself

for o in all_meths: setattr(APIRoute, o, functools.partialmethod(APIRoute, methods=o))

ar = APIRoute


In [31]:
class TestAR(RouteProvider):
    def __init__(self, a): self.a = a

    @APIRoute('/hi')
    @staticmethod
    def hi(): return f'hi there'

    @ar('/yoyo')
    def yoyo(self): 
        return f'a yoyo called {self.a}'

    @ar
    def foo(self): 
        return f'foo {self.a}'


t = TestAR('yo')
test_eq(t.yoyo(), 'a yoyo called yo')

app, cli, _ = get_app()
t.ar.to_app(app)

test_eq(cli.get('/hi').text, 'hi there')
test_eq(cli.get('/yoyo').text, 'a yoyo called yo')
test_eq(cli.get('/foo').text, 'foo yo')

test_eq(t.ar.yoyo.to(), '/yoyo')


If you don't need to initialize APIRouterD (e.g. `prefix`), or would set them afterwards, or you mind defining the `APIRouterD` attribute explicitly, use `APIRoute` or the `ar` shortcut.


If like me, you don't like stupid wiggly reds <span style="color:red">~~~</span>, inherit from `RouteProvider`.

In [32]:
# type: ignore

class A(RouteProvider):
    @ar
    def value(self): return self._value

class B(A):
    @ar.post
    def value(self, value:int=None):
        if value is not None: self._value = value
        return self._value

@dataclass
class C(B):
    _value:int = 1


app, cli, _ = get_app()

c = C()
c.ar.to_app(app)

test_eq(cli.get('/value').text, '1')
test_eq(cli.post('/value?value=5').text, '5')
test_eq(c.value(), 5)


# Adding routes to the app
> Convenience functions for registering routes from providers into FastHTML apps.


In [33]:
#| export

def add_routes(self: FastHTML, 
        prov:APIRouter|RouteProviderP,  # APIRouter or RouteProvider
        mount:bool=False,               # If True, mount routes under `path`
        path:str|None=None,             # if mount, Base path or APIRouter prefix
        name:str|None=None,             # if mount, Optional name for the routes group
        appcls:Callable=nb_app          # if mount, use factory
    ) -> APIRouter:
    "Register routes from a provider into a FastHTML app"
    ar = prov if isinstance(prov, APIRouter) else prov.ar  # TODO: search for the class attribute actually used
    rapp = appcls() if mount else self
    ar.to_app(rapp)
    if mount:
        ar.__routename__ = name or new_id(prov)
        if not path: path = ar.prefix or f"/{ar.__routename__}"
        if not ar.prefix: ar.prefix = path
        self.mount(path, rapp, name=ar.__routename__)
    return ar


`add_routes` handles two scenarios:
1. `mount` is `False`. This is equivalent to `APIRouter.to_app`, the routes are available under `APIRuter.prefix`.
2. `mount` is `True`. The routes are mounted under `path` and are available under path/prefix.

App routes can be organized at two levels:
1. **root Level**: Global routes
2. **Provider Level**: Scoped routes defined by providers

This allows for clean organization of routes and natural encapsulation of component behavior directly in the notebook. This will be handy when we start defining ipywidgets like widgets with FastHTML.

Though I think it's convenient, you don't need to mount routes providers at all, add its routes directly to the root level app.


In [34]:
class Counter(RouteProvider):
    def __init__(self, value:int=0): self._value = value
    
    @ar('/value')
    def get_value(self): 
        return self._value  # `0` is not a valid FastHTML response
    
    @ar('/inc')
    def increment(self): 
        self._value += 1
        return self.get_value()


app, cli, _ = get_app()

counter = Counter()
add_routes(app, counter)

test_eq(cli.get('/inc').text, '1')


Add routes to current app. Routes available under `{self.ar.prefix}/...` or `/{self.ar.to()}/...`


In [35]:
#| export

def mount(app: FastHTML, prov:APIRouter|RouteProviderP, path:str|None=None, name:str|None=None):
    return add_routes(app, prov, True, path, name)


Use `mount` to define routes providers with instances. In this way we can have notebook level routes, those defined with global `app.route`, and scoped routes defined with providers and `add_routes` or `mount`.

`mount` and `add_routes` return the APIRouter used to mount the routes.  


In [36]:
counter = Counter(3)
mount(app, counter, '/counter')  # routes under /counter/..., and root.

test_eq(cli.get('/inc').text, '2')

test_eq(cli.get('/counter/value').text, '3')
test_eq(cli.get('/counter/inc').text, '4')

test_eq(app.routes[-1].name, counter.ar.name())


Mount with explicit path.  
An APIRouter from an instance, discloses its mounted name with ...`.ar.name()`.


In [37]:
counter = Counter(-3)
car = mount(app, counter)  # routes under /counter/..., and root.

test_eq(cli.get(f"/{car.to()}/value").text, '-3')
test_eq(cli.get(f"/{car.to()}/inc").text, '-2')

test_eq(cli.get(app.url_path_for(car.name('increment'))).text, '-1')


Mount routes with automatic path generation.


# Route introspection


In [38]:
class Counter(RouteProvider):
    def __init__(self): self._value = 0
    
    @ar('/value')
    def get_value(self): 
        return self._value  # `0` is not a valid FastHTML response
    
    @ar('/inc')
    def increment(self): 
        self._value += 1
        return self.get_value()

    @ar('/add/{x}')
    def add(self, x:int): 
        self._value += x
        return self.get_value()

    @ar.post('/sub')
    def sub(self, x:int): 
        self._value -= x
        return self.get_value()
    
    @ar
    def link(self, req): 
        nm = self.ar.name('add')
        req.url_for(f"{nm}", x=2)
        return (Div('+ 3', link=uri(nm, x='3')), 
                Div('+ 2', href=f"{req.url_for(f"{nm}", x=2)}"))


app, cli, rt = get_app()

c = Counter()
test_eq(c.increment(), 1)

c.ar.to_app(app)

test_eq(c.ar.name(), '')

test_eq(cli.get('/inc').text, '2')
test_eq(cli.get('/add/5').text, '7')
test_eq(cli.post('/sub', data={'x':2}).text, '5')
test_eq(cli.post('/sub?x=3').text, '2')
test_eq(cli.get('/link').text, ' <div href="/add/3">+ 3</div>\n <div href="http://nb/add/2">+ 2</div>\n')

test_eq(c.increment(), 3)


In [39]:
test_eq(c.ar.increment.to(), '/inc')
test_eq(c.ar.add.to(), '/add/{x}')
test_eq(c.ar.to('add'), '/add/{x}')
test_eq(c.ar.to('sub', x=3), '/sub?x=3')

test_eq(app.url_path_for(c.ar.name('increment')), '/inc')
test_eq(app.url_path_for(c.ar.name('add'), x=5), '/add/5')
test_eq(app.url_path_for(c.ar.name('sub')), '/sub')


In [40]:
app, cli, rt = get_app()

c = Counter()
mount(app, c, '/counter')

test_eq(c.ar.prefix, '/counter')

test_eq(c.ar.add.to(), '/add/{x}')
test_eq(c.ar.to('increment'), 'counter/inc')
test_eq(c.ar.to('add'), 'counter/add/{x}')
test_eq(c.ar.to('sub', x=3), 'counter/sub?x=3')

test_eq(app.url_path_for(c.ar.name('link')), '/counter/link')

test_eq(c.ar.name('increment'), f"{c.ar.name()}:Counter_increment")
test_eq(app.url_path_for(c.ar.name('increment')), '/counter/inc')
test_eq(app.url_path_for(c.ar.name('add'), x=5), '/counter/add/5')


In [41]:
app, cli, rt = get_app()

c = Counter()
mount(app, c, '/counter', 'cnt')

test_eq(app.url_path_for(c.ar.name('link')), '/counter/link')

test_eq(c.ar.name('increment'), f"cnt:Counter_increment")
test_eq(app.url_path_for(c.ar.name('increment')), '/counter/inc')
test_eq(app.url_path_for(c.ar.name('add'), x=5), '/counter/add/5')


More examples:

In [42]:
app, cli, rt = get_app()

@app.get("/hostie")
def show_host(req): return req.headers['host']

test_eq(cli.get('/hostie').text, 'nb')


@rt('/user/{nm}', name='gday')
def get(nm:str=''): return f"Good day to you, {nm}!"

test_eq(cli.get('/user/Jeremy').text, 'Good day to you, Jeremy!')


@rt('/link')
def get(req): return f"{req.url_for('gday', nm='Alexis')}; {req.url_for('show_host')}"

test_eq(cli.get('/link').text, 'http://nb/user/Alexis; http://nb/hostie')


In [43]:
class _User:
    @ar('/link/{nm}')
    def get(self, req, nm:str=''): 
        return f"{req.url_for('gday', nm=nm)}; {req.url_for('show_host')}"

u = _User()
u.ar.to_app(app)

test_eq(cli.get('/link/Vic').text, 'http://nb/user/Vic; http://nb/hostie')


`APIRouterD` is useful when you want to split your application routes across multiple `.py` files or routes providers that are part of a single FastHTMl application. It accepts an optional `prefix` argument that will be applied to all routes within that instance of `APIRouterD`.

Below we define several hypothetical product related routes in a `Products` and then demonstrate how they can seamlessly be incorporated into a FastHTML app instance.

In [44]:
class Products:
    ar = APIRouterD('/products')

    @ar('/all')
    def all_products(self, req):
        return Div(
            "Welcome to the Products Page! Click the button below to look at the details for product 42",
            Div(
                Button(
                    'Details',
                    hx_get=req.url_for('details', pid=42),
                    hx_target='#products_list',
                    hx_swap='outerHTML',
                ),
            ),
            id='products_list',
        )


    @ar('/{pid}', name='details')  # or @ar('/{pid}') or @ar.get('/{pid}') or @ar.get('/{pid}', name='details')
    def details(self, pid: int):
        return f"Here are the product details for ID: {pid}"


app, cli, rt = get_app()

products = Products()
products.ar.to_app(app)

print(str(products.ar.all_products.to()))
print(str(products.ar.rt_funcs.details))


/products/all
/products/{pid}


Since we specified the `prefix=/products` in our hypothetical object, all routes defined in that provider will be found under `/products`.

In [45]:
@rt
def index():
    return Div(
        "Click me for a look at our products",
        hx_get=products.ar.all_products,  # or Products.ar.all_products or Products/products.ar.to('all_products')
        hx_swap="outerHTML",
    )

cli.get('/').text


' <div hx-get="/products/all" hx-swap="outerHTML">Click me for a look at our products</div>\n'

In [46]:
cli.get('/products/all').text

' <div id="products_list">\nWelcome to the Products Page! Click the button below to look at the details for product 42   <div>\n<button hx-get="http://nb/products/42" hx-swap="outerHTML" hx-target="#products_list">Details</button>   </div>\n </div>\n'

Note how you can reference our python route functions via `APIRouter.rt_funcs` or `APIRouter.{name}` or `APIRouter.to('name')` in your `hx_{http_method}` calls like normal.

# Colophon
----


In [47]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [53]:
if FC.IN_NOTEBOOK:
    nb_path = '30_route_provider.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
